<h2>This notebook will be mainly used for the capstone project</h2>

In [295]:
#https://github.com/mtk12/IBM-Data-science-capstone-project/blob/master/Capstone-Project.ipynb
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format

import json # library to handle JSON files

import geocoder as geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [296]:
#CSV from https://www.kaggle.com/anthonypino/melbourne-housing-market#Melbourne_housing_FULL.csv
df = pd.read_csv('Melbourne_housing_FULL.csv')
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,nan,SS,Jellis,3/09/2016,2.50,3067.00,2.00,1.00,1.00,126.00,nan,nan,Yarra City Council,-37.80,145.00,Northern Metropolitan,4019.00
1,Abbotsford,85 Turner St,2,h,1480000.00,S,Biggin,3/12/2016,2.50,3067.00,2.00,1.00,1.00,202.00,nan,nan,Yarra City Council,-37.80,145.00,Northern Metropolitan,4019.00
2,Abbotsford,25 Bloomburg St,2,h,1035000.00,S,Biggin,4/02/2016,2.50,3067.00,2.00,1.00,0.00,156.00,79.00,1900.00,Yarra City Council,-37.81,144.99,Northern Metropolitan,4019.00
3,Abbotsford,18/659 Victoria St,3,u,nan,VB,Rounds,4/02/2016,2.50,3067.00,3.00,2.00,1.00,0.00,nan,nan,Yarra City Council,-37.81,145.01,Northern Metropolitan,4019.00
4,Abbotsford,5 Charles St,3,h,1465000.00,SP,Biggin,4/03/2017,2.50,3067.00,3.00,2.00,0.00,134.00,150.00,1900.00,Yarra City Council,-37.81,144.99,Northern Metropolitan,4019.00


In [297]:
df.shape

(34857, 21)

In [299]:
df = df[['Suburb', 'Price', 'Postcode', 'Lattitude', 'Longtitude']]
df['Neighbourhood'] = df[['Suburb']]
df.head()

,Suburb,Price,Postcode,Lattitude,Longtitude,Neighbourhood
0,Abbotsford,nan,3067.00,-37.80,145.00,Abbotsford
1,Abbotsford,1480000.00,3067.00,-37.80,145.00,Abbotsford
2,Abbotsford,1035000.00,3067.00,-37.81,144.99,Abbotsford
3,Abbotsford,nan,3067.00,-37.81,145.01,Abbotsford
4,Abbotsford,1465000.00,3067.00,-37.81,144.99,Abbotsford


In [308]:
df = df.dropna()
df.reset_index(drop=True)
df = pd.DataFrame(df.groupby(['Suburb'], sort = False).mean())
df.head()

,Price,Postcode,Lattitude,Longtitude
Suburb,,,,
Abbotsford,1096603.90,3067.00,-37.80,145.00
Airport West,780529.42,3042.00,-37.72,144.88
Albert Park,1983664.71,3206.00,-37.84,144.95
Alphington,1441155.56,3078.00,-37.78,145.03
Altona,872917.93,3018.00,-37.87,144.82


In [309]:
ls = []

for index, row in df.iterrows():
    ls.append((index, row['Postcode'], row['Price'], row['Lattitude'], row['Longtitude']))
data = pd.DataFrame(ls, columns = ('Suburb', 'Postcode', 'AvgPrice', 'Lattitude', 'Longtitude'))
data.head()

,Suburb,Postcode,AvgPrice,Lattitude,Longtitude
0,Abbotsford,3067.00,1096603.90,-37.80,145.00
1,Airport West,3042.00,780529.42,-37.72,144.88
2,Albert Park,3206.00,1983664.71,-37.84,144.95
3,Alphington,3078.00,1441155.56,-37.78,145.03
4,Altona,3018.00,872917.93,-37.87,144.82


In [310]:
address = 'Melbourne, Victoria'

geolocator = Nominatim(user_agent="melbourne_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Melbourne are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Melbourne are -37.8142176, 144.9631608.


In [313]:
# create map of Melbourne using latitude and longitude values
map_melbourne = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, suburb in zip(data['Lattitude'], data['Longtitude'], data['Suburb']):
    label = '{}, {}'.format(data, suburb)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_melbourne)  
    
map_melbourne

In [314]:
CLIENT_ID = 'TGMPXAQXAHIPIEVB2Q2GKVVNJ34L5KX4Y5IQVQEV5JBLBHUR' # your Foursquare ID
CLIENT_SECRET = 'E3YE5OPQ0BU4HQUWNQSYLJ4EQX0W5HBDYV5XEYKLWAVFEFOR' # your Foursquare Secret
VERSION = '20180604'

In [315]:
def getBuiltUrl(neigh_lat,neigh_long,radius=1400):
    # type your answer here
    LIMIT=100
    #radius=1000
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_lat, 
    neigh_long, 
    radius, 
    LIMIT)
    return url

In [317]:
data.iloc[0]

Suburb        Abbotsford
Postcode         3067.00
AvgPrice      1096603.90
Lattitude         -37.80
Longtitude        145.00
Name: 0, dtype: object

In [318]:
neigh_name, neigh_post, neigh_price, neigh_lat, neigh_long = data.iloc[0]
print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_lat, 
                                                               neigh_long))
results = requests.get(getBuiltUrl(neigh_lat,neigh_long)).json()
results

Latitude and longitude values of Abbotsford are -37.8040351948052, 144.99731766233762.


{'meta': {'code': 200, 'requestId': '5dedfa8bc9497946ca41bf44'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b5ccacbf964a520424429e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16d941735',
         'name': 'Café',
         'pluralName': 'Cafés',
         'primary': True,
         'shortName': 'Café'}],
       'id': '4b5ccacbf964a520424429e3',
       'location': {'address': '60 Nicholson St',
        'cc': 'AU',
        'city': 'Abbotsford',
        'country': 'Australia',
        'distance': 370,
        'formattedAddress': ['60 Nicholson St',
         'Abbotsford VIC 3067',
         'Australia'],
        'labeledLatLngs': [{'label': 'display',
          'lat': -37.8073178966101,
          

In [319]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [320]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Three Bags Full,Café,-37.81,145.00
1,Lentil As Anything,Vegetarian / Vegan Restaurant,-37.80,145.00
2,Stomping Ground Brewery & Beer Hall,Brewery,-37.80,144.99
3,Au79,Café,-37.81,145.00
4,Laird Hotel,Gay Bar,-37.81,144.99


In [326]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
data.head()

100 venues were returned by Foursquare.


,Suburb,Postcode,AvgPrice,Lattitude,Longtitude
0,Abbotsford,3067.00,1096603.90,-37.80,145.00
1,Airport West,3042.00,780529.42,-37.72,144.88
2,Albert Park,3206.00,1983664.71,-37.84,144.95
3,Alphington,3078.00,1441155.56,-37.78,145.03
4,Altona,3018.00,872917.93,-37.87,144.82


In [322]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        LIMIT=100
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Groups error, need to try another time

In [324]:
melbourne_venues = getNearbyVenues(names=data['Suburb'], 
                                   latitudes=data['Lattitude'], 
                                   longitudes=data['Longtitude'], 
                                   radius=500)

Abbotsford
Airport West
Albert Park
Alphington
Altona
Altona North
Armadale
Ascot Vale
Ashburton
Ashwood
Avondale Heights
Balaclava
Balwyn


KeyError: 'groups'

In [114]:
print(melbourne_venues.shape)
melbourne_venues.head()

(18783, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Melbourne,-37.8111,144.9715,San Telmo,-37.812300,144.972378,Argentinian Restaurant
1,Melbourne,-37.8111,144.9715,Lupino,-37.812774,144.972311,Italian Restaurant
2,Melbourne,-37.8111,144.9715,Gelateria Primavera,-37.811045,144.972824,Ice Cream Shop
3,Melbourne,-37.8111,144.9715,Longrain Restaurant & Bar,-37.810733,144.971147,Thai Restaurant
4,Melbourne,-37.8111,144.9715,Hill of Content,-37.811863,144.970760,Bookstore


In [118]:
melbourne_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
East Melbourne,706,706,706,706,706,706
Melbourne,4684,4684,4684,4684,4684,4684
North Melbourne,2894,2894,2894,2894,2894,2894
Port Melbourne,5323,5323,5323,5323,5323,5323
South Melbourne,3954,3954,3954,3954,3954,3954
West Melbourne,1222,1222,1222,1222,1222,1222


In [119]:
print('There are {} uniques categories.'.format(len(melbourne_venues['Venue Category'].unique())))

There are 222 uniques categories.


In [120]:
# one hot encoding
melbourne_onehot = pd.get_dummies(melbourne_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
melbourne_onehot['Neighborhood'] = melbourne_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [melbourne_onehot.columns[-1]] + list(melbourne_onehot.columns[:-1])
london_onehot = melbourne_onehot[fixed_columns]

melbourne_onehot.head()

,African Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Stop,Café,Candy Store,Cantonese Restaurant,Casino,Caucasian Restaurant,Chaat Place,Chinese Restaurant,Chocolate Shop,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Comedy Club,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Night Market,Nightclub,Noodle House,Paella Restaurant,Paintball Field,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,River,Road,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Art,Strip Club,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Neighborhood
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Melbourne
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [121]:
melbourne_onehot.shape

(18783, 223)

In [122]:
melbourne_grouped = melbourne_onehot.groupby('Neighborhood').mean().reset_index()
melbourne_grouped

,Neighborhood,African Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Stop,Café,Candy Store,Cantonese Restaurant,Casino,Caucasian Restaurant,Chaat Place,Chinese Restaurant,Chocolate Shop,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Comedy Club,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Night Market,Nightclub,Noodle House,Paella Restaurant,Paintball Field,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,River,Road,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Art,Strip Club,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,East Melbourne,0.000000,0.000000,0.000000,0.00000,0.002833,0.001416,0.002833,0.011331,0.011331,0.021246,0.002833,0.000000,0.002833,0.014164,0.000000,0.000000,0.000000,0.000000,0.001416,0.000000,0.000000,0.000000,0.000000,0.005666,0.000000,0.00000,0.000000,0.015581,0.000000,0.000000,0.000000,0.000000,0.002833,0.000000,0.160057,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.015581,0.049575,0.000000,0.000000,0.002833,0.000000,0.000000,0.000000,0.035411,0.000000,0.000000,0.043909,0.000000,0.000000,0.008499,0.000000,0.002833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007082,0.000000,0.002833,0.000000,0.000000,0.008499,0.000000,0.000000,0.000000,0.00000,0.000000,0.011331,0.011331,0.000000,0.000000,0.002833,0

In [127]:
a=melbourne_grouped
num_top_venues = 5

for hood in melbourne_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = a[a['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----East Melbourne----
            venue  freq
0            Café  0.16
1           Hotel  0.08
2        Wine Bar  0.05
3     Coffee Shop  0.05
4  Cricket Ground  0.04


----Melbourne----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.07
2    Korean Restaurant  0.05
3                  Bar  0.04
4  Japanese Restaurant  0.03


----North Melbourne----
                venue  freq
0                Café  0.25
1                 Pub  0.04
2  Light Rail Station  0.04
3                 Bar  0.04
4   Korean Restaurant  0.03


----Port Melbourne----
                 venue  freq
0                 Café  0.20
1    Fish & Chips Shop  0.05
2                  Pub  0.05
3  Japanese Restaurant  0.04
4               Bakery  0.04


----South Melbourne----
         venue  freq
0         Café  0.28
1          Pub  0.07
2  Coffee Shop  0.05
3          Bar  0.03
4    Gastropub  0.03


----West Melbourne----
            venue  freq
0            Café  0.24
1             Pub  0.

In [128]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [129]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = a['Neighborhood']

for ind in np.arange(a.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(a.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Melbourne,Café,Hotel,Coffee Shop,Wine Bar,Cricket Ground,Thai Restaurant,Convenience Store,Train Station,Light Rail Station,Italian Restaurant
1,Melbourne,Café,Coffee Shop,Korean Restaurant,Bar,Cocktail Bar,Japanese Restaurant,Italian Restaurant,Hotel,Chinese Restaurant,Burger Joint
2,North Melbourne,Café,Bar,Light Rail Station,Pub,Coffee Shop,Italian Restaurant,Convenience Store,Grocery Store,Korean Restaurant,Malay Restaurant
3,Port Melbourne,Café,Pub,Fish & Chips Shop,Bakery,Japanese Restaurant,Gym,Breakfast Spot,Light Rail Station,Italian Restaurant,Thai Restaurant
4,South Melbourne,Café,Pub,Coffee Shop,Bar,Gastropub,Japanese Restaurant,Thai Restaurant,Grocery Store,Asian Restaurant,Light Rail Station


In [130]:
neighborhoods_venues_sorted.rename(columns = {'Neighborhood':'Location'}, inplace = True)

In [138]:
a.head()

,Neighborhood,African Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Stop,Café,Candy Store,Cantonese Restaurant,Casino,Caucasian Restaurant,Chaat Place,Chinese Restaurant,Chocolate Shop,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Comedy Club,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Night Market,Nightclub,Noodle House,Paella Restaurant,Paintball Field,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,River,Road,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Art,Strip Club,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Price
0,East Melbourne,0.000000,0.000000,0.000000,0.00000,0.002833,0.001416,0.002833,0.011331,0.011331,0.021246,0.002833,0.000000,0.002833,0.014164,0.000000,0.000000,0.000000,0.000000,0.001416,0.000000,0.000000,0.000000,0.000000,0.005666,0.000000,0.00000,0.000000,0.015581,0.000000,0.000000,0.0,0.000000,0.002833,0.0,0.160057,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.015581,0.049575,0.000000,0.000000,0.002833,0.000000,0.000000,0.000000,0.035411,0.000000,0.000000,0.043909,0.000000,0.000000,0.008499,0.000000,0.002833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007082,0.000000,0.002833,0.000000,0.000000,0.008499,0.000000,0.000000,0.000000,0.00000,0.000000,0.011331,0.011331,0.000000,0.000000,0.002833,0.000